In [2]:
# spray_simple.ipynb
# simple spray deposition control program
# written by Veit Wagner
# 07.07.2023 v0.01 initial version
# =========================================================
#      load libraries etc
# =========================================================
import numpy as np
from datetime import datetime
import time
import os
from instr.asrl_pyserial import asrl_serial_class
import minimalmodbus                      # for Präzitherm TR28-3T-RS hotplate (instr/cal3300.py),   https://minimalmodbus.readthedocs.io/en/stable/
from instr.dug2005 import dug2005_class   # Sonozap Digital Ultrasonic Generator (instr/dug2005.py)
from instr.al1010 import al1010_class     # Aladdin pump controller AL-1010 (instr/al1010.py)
from instr.cal3300 import cal3300_class   # Präzitherm TR28-3T-RS hotplate (instr/cal3300.py)
from instr.pd4n5918 import pd4n5918_class # stepper motor PD4-N5918 M4204 from NanoTec (instr/pd4n5918.py)
# --- wide jupyter window display ---
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# --- set is windows flag ---
is_win = os.name == 'nt'
# --- show available ports ---
print("--- list of found serial ports ---")
import serial.tools.list_ports as list_ports
ports = list_ports.comports()
for port, desc, hwid in sorted(ports):
    print(f"{port}: {desc} [{hwid}]")
    

--- list of found serial ports ---
COM1: Kommunikationsanschluss (COM1) [ACPI\PNP0501\0]
COM5: Silicon Labs CP210x USB to UART Bridge (COM5) [USB VID:PID=10C4:EA60 SER=0001 LOCATION=1-4.2]
COM6: USB Serial Port (COM6) [USB VID:PID=0403:6001 SER=FTSXNG08A]
COM7: USB Serial Port (COM7) [USB VID:PID=0403:6001 SER=FTG4T3HRA]
COM8: Silicon Labs CP210x USB to UART Bridge (COM8) [USB VID:PID=10C4:EA60 SER= LOCATION=1-4.1.2]


In [3]:
# =========================================================
#      initialization of instruments
# =========================================================
use_du2 = True #True                # Sonozap Digital Ultrasonic Generator (instr/dug2005.py)
use_al1 = True #True                # Aladdin pump controller AL-1010 (instr/al1010.py)
use_c33 = True #True                # Präzitherm TR28-3T-RS hotplate (instr/cal3300.py)
use_pd4 = True                # stepper motor PD4-N5918 M4204 from NanoTec (instr/pd4n5918.py)


if use_du2:
    asrl_no_du2 = 7 # (COM7) [USB VID:PID=0403:6001 SER=FTG4T3HRA]
    asrl = asrl_serial_class(asrl_no=asrl_no_du2, baud=38400, data_bits=8, parity=0, stop_bits=10, flow_ctrl=0, termchar_en=0)
    asrl.set_timeout(0.1) # sec
    du2 = dug2005_class(asrl)
    print(f'--- Sonozap Digital Ultrasonic Generator (COMport {asrl_no_du2}) ---')
    print('Ping:', du2.ping())
    print('Software ver:', du2.get_softw_ver())
    print('System state:', du2.get_system_state())

if use_al1:
    asrl_no_al1 = 6  # COM6: #com='/dev/ttyUSB2'  #com='COM6' # (COM6) [USB VID:PID=0403:6001 SER=FTSXNG08A]
    asrl = asrl_serial_class(asrl_no=asrl_no_al1, baud=19200, data_bits=8, parity=0, stop_bits=10, flow_ctrl=0, termchar_en=1, termchar=13)
    asrl.set_timeout(0.05)
    al1 = al1010_class(asrl, adr=1)
    print(f'--- Aladdin pump controller AL-1010 (COMport {asrl_no_al1}) ---')
    print( f"Version of pump {al1.get_dflt_adr()} : {al1.firmware_version()}" )
    
if use_c33:
    asrl_no_c33 = 8  # COM6: #com='/dev/ttyUSB2'  #com='COM6' # (COM8) [USB VID:PID=10C4:EA60 SER= LOCATION=1-4.1.2]
    com     = f"COM{asrl_no_c33:d}" # Windows
    adr     = 1  #slave address
    mdbs = minimalmodbus.Instrument(com, adr, mode='rtu') # port name, slave address (in decimal)
    mdbs.serial.baudrate = 19200 # baud, for more parameters see https://minimalmodbus.readthedocs.io/en/stable/usage.html
    c33 = cal3300_class(mdbs, adr=adr)
    typ, txt = c33.get_model()
    print('--- Präzitherm TR28-3T-RS hotplate (instr/cal3300.py) (COMport {asrl_no_c33}) ---') 
    print( f"Version of T ctrl : '{txt}' (0x{typ:02X})" )

if use_pd4:
    asrl_no_pd4 = 5  # COM5: #com='/dev/ttyUSB2'  #com='COM5' # (COM5) [USB VID:PID=10C4:EA60 SER=0001 LOCATION=1-4.1.2]
    devser   = f"COM{asrl_no_pd4:d}" if is_win else '/dev/ttyUSB0'
    pd4_mids = [1] #[2] # list of motors
    asrl     = asrl_serial_class(devser=devser, baud=115200, data_bits=8, parity=0, stop_bits=10, flow_ctrl=2, termchar_en=1, termchar=13)
    pd4      = pd4n5918_class(asrl, mids=pd4_mids)
    print(f'--- stepper motor PD4-N5918 M4204 from NanoTec (instr/pd4n5918.py) (COMport {asrl_no_pd4}) ---')
    mid = pd4_mids[0] # use 1st motor in list, which has 0.001 mm per step
    pd4.set_mid_and_mm_per_step(mid=mid, mm_per_step=0.001)
    print(f"motor {mid} status : {pd4.Status_auslesen(mid):X} ,  position : {pd4.get_current_pos_abs_mm()} mm")    

--- Sonozap Digital Ultrasonic Generator (COMport 7) ---
Ping: (b'', 0)
Software ver: 5.01
System state: 1
--- Aladdin pump controller AL-1010 (COMport 6) ---
Version of pump 1 : NE1010V3.922
--- Präzitherm TR28-3T-RS hotplate (instr/cal3300.py) (COMport {asrl_no_c33}) ---
Version of T ctrl : '33/9300 SSD / SSD' (0x102)
--- stepper motor PD4-N5918 M4204 from NanoTec (instr/pd4n5918.py) (COMport 5) ---
motor 1 status : A1 ,  position : 0.0 mm


In [16]:
# =========================================================
#      deposition run with given parameters
# =========================================================
par = {
    'user':                    'TS,VW',
    'sample':                  'test',
    'comment':                 'test',
    'start date':               datetime.now().strftime("%d.%m.%Y %H:%M:%S %z"),
    'sonic_power_perc':              50,
    'syring_diameter_mm':           20,
    'syring_pump_rate_mL_min':      0.5,
    'syring_pump_start_delay_s':    0.5,  # [sec] delay of pump start after motor start
    'syring_pump_depos_time_s':    20.0,  # [sec] pump active time pumping
    'temperature_sample_C':        40.0,
    'nozzle_run_no_of_repeats':       1,  # 1=single run only
    'nozzle_run_period_s':         20.0,  # [sec] time from nozzle start to previous repeat nozzle start (only needed if not single run)
    'nozzle_run_both_directions':  False,  # flag if deposition on both directions (only needed if not single run)
    'nozzle_pos_start_mm':        150,
    'nozzle_pos_end_mm':          5.0,
    'nozzle_pos_speed_depos_mm_s':  5.0,
    'nozzle_pos_speed_mm_s':       15.0,
    'nozzle_pos_acc_mm_s2':         5.0,
}
t00 = time.time()
print(par)
print()

# --- set temperature ---
if use_c33:
    print( f"{time.time()-t00:7.3f}s  setting temperature setpoint to : {par['temperature_sample_C']} C  ;  current temperature : {c33.readT_C()} C" )
    c33.set_setpoint(par['temperature_sample_C']) # set target T in C
    for i in range(100000):
        if abs(c33.readT_C() - par['temperature_sample_C']) < 0.5:
            break
        time.sleep(0.5)
    print( f"{time.time()-t00:7.3f}s  temperature set:  setpoint      : {c33.read_setpoint()} C  ;  current temperature : {c33.readT_C()} C" )

# --- start ultrasonic nozzle ---
if use_du2:
    du2.set_system_state(du2.running)
    print(f"{time.time()-t00:7.3f}s  Ultrasonic nozzle System state: {du2.get_system_state()}")
    time.sleep(1)
    du2.set_power_level( min(100, max(0, int(round(par['sonic_power_perc'])) ) ) )
    time.sleep(0.3)
    print(f"{time.time()-t00:7.3f}s  Ultrasonic nozzle Power level :  target: {par['sonic_power_perc']} % , current value: {du2.get_power_level()} % {du2.get_power()} W  ; Frequency: {du2.get_frequency()} Hz")

# --- set syring parameter ---
if use_al1:
    al1.stop_pump() # stop pumping
    al1.set_diameter(par['syring_diameter_mm']) # [mm] diameter
    al1.set_pumping_rate(par['syring_pump_rate_mL_min'], 'MM') # allowed units: 'UM'=uL/min, 'MM'=mL/min, 'UH'=uL/hr, 'MH'=mL/hr
    al1.set_volume(0) # 0=cont. pumping
    al1.set_pumping_direction(al1.infuse) # =al1.infuse, al1.withdraw, al1.reverse

# --- move nozzle to start position ---
if use_pd4:
    print(f"{time.time()-t00:7.3f}s  Moving to start position: {par['nozzle_pos_start_mm']} mm")
    pd4.move_to_pos_abs_mm(par['nozzle_pos_start_mm'], par['nozzle_pos_speed_mm_s'], par['nozzle_pos_acc_mm_s2'])
    while not pd4.ready(): # wait finish movement
            time.sleep(0.02)


# === repetition loop ===
for i_rep in range(par['nozzle_run_no_of_repeats']):
    print(f"{time.time()-t00:7.3f}s  --- Repetition i_rep = {i_rep} ---")

    # --- move nozzle to start position ---
    if use_pd4 and ((not par['nozzle_run_both_directions']) and (i_rep > 0)):
        pd4.move_to_pos_abs_mm(par['nozzle_pos_start_mm'], par['nozzle_pos_speed_mm_s'], par['nozzle_pos_acc_mm_s2'])
        while not pd4.ready(): # wait finish movement
                time.sleep(0.02)

    # --- wait for deposition start ---
    if (i_rep == 0) or (time.time() > t_n):
        t_n = time.time()    # remember time of initial start or if given period too short 
    else:
        while time.time() < t_n: # wait finish movement
                time.sleep(0.001)
        
    print(f"{time.time()-t00:7.3f}s  use t_n-t00 = {t_n-t00} s")

    # --- move nozzle for deposition ---
    if use_pd4:
        pos = par['nozzle_pos_end_mm'] if (not par['nozzle_run_both_directions'] ) or (i_rep % 2 == 0)  else  par['nozzle_pos_start_mm']
        pd4.move_to_pos_abs_mm(pos, par['nozzle_pos_speed_depos_mm_s'], par['nozzle_pos_acc_mm_s2'])
        print(f"{time.time()-t00:7.3f}s  Start deposition movement from {pd4.get_current_pos_abs_mm()} mm towards {pos} mm")

    if use_al1:
        # --- wait for syring start ---
        t_x = t_n + par['syring_pump_start_delay_s']
        while time.time() < t_x: # wait finish mov
            time.sleep(0.001)
        al1.start_pump()
        t_tmp = time.time()-t00
        s = f" , current motorposition {pd4.get_current_pos_abs_mm()} mm" if use_pd4  else ""
        print(f"{t_tmp:7.3f}s  Start pump{s}")
        # --- wait for syring stop ---
        t_x = t_x + par['syring_pump_depos_time_s']
        while time.time() < t_x: # wait finish mov
            time.sleep(0.001)
        al1.stop_pump()
        t_tmp = time.time()-t00
        s = f" , current motorposition {pd4.get_current_pos_abs_mm()} mm" if use_pd4  else ""
        print(f"{t_tmp:7.3f}s  Stop pump{s}")
        
    if use_pd4:
        while not pd4.ready(): # wait finish movement
            time.sleep(0.02)
    
    # --- set start time for next repetition ---
    t_n += par['nozzle_run_period_s'] 
        
# === end of repetition loop ===

# --- move nozzle back to start position ---
if use_pd4:
    pd4.move_to_pos_abs_mm(par['nozzle_pos_start_mm'], par['nozzle_pos_speed_mm_s'], par['nozzle_pos_acc_mm_s2'])
    while not pd4.ready(): # wait finish movement
            time.sleep(0.02)

if use_al1:
    al1.stop_pump() # stop pumping , for safety

if use_du2:
    du2.set_system_state(du2.stopped)

print(f"{time.time()-t00:7.3f}s  finished.")

{'user': 'TS,VW', 'sample': 'test', 'comment': 'test', 'start date': '07.08.2023 13:25:41 ', 'sonic_power_perc': 50, 'syring_diameter_mm': 20, 'syring_pump_rate_mL_min': 1.5, 'syring_pump_start_delay_s': 0.5, 'syring_pump_depos_time_s': 20.0, 'temperature_sample_C': 40.0, 'nozzle_run_no_of_repeats': 1, 'nozzle_run_period_s': 20.0, 'nozzle_run_both_directions': False, 'nozzle_pos_start_mm': 150, 'nozzle_pos_end_mm': 5.0, 'nozzle_pos_speed_depos_mm_s': 5.0, 'nozzle_pos_speed_mm_s': 15.0, 'nozzle_pos_acc_mm_s2': 5.0}

  0.000s  setting temperature setpoint to : 40.0 C  ;  current temperature : 40.1 C
  0.285s  temperature set:  setpoint      : 40.0 C  ;  current temperature : 40.1 C
  0.392s  Ultrasonic nozzle System state: 2
  1.739s  Ultrasonic nozzle Power level :  target: 50 % , current value: 55 % 1.393 W  ; Frequency: 127000 Hz
 W: stop_pump(01) -> 'S?NA'
  1.884s  Moving to start position: 150 mm
  1.959s  --- Repetition i_rep = 0 ---
  1.959s  use t_n-t00 = 1.9591472148895264 s
  

In [17]:
# =========================================================
#      close instruments
# =========================================================
if use_du2:
    du2.asrl.close()

if use_al1:
    al1.asrl.close()
    
if use_c33:
    c33.mdbs.serial.close()

if use_pd4:
    pd4.asrl.close()

In [ ]:
# =========================================================
#      interactive operation of Sonozap Digital Ultrasonic Generator (instr/dug2005.py)
# =========================================================
print('set System state running.')
du2.set_system_state(du2.running)
print('System state:', du2.get_system_state())

time.sleep(1)
print('Frequency (Hz) :', du2.get_frequency())
print('Power     (W)  :', du2.get_power())
plvl = du2.get_power_level()
print('Power level (%):', plvl)
time.sleep(1)
print('set power level')
du2.set_power_level( 80) #int(min(100, max(0, plvl-5))) )
time.sleep(0.3)
print('Power level (%):', du2.get_power_level())
time.sleep(1)

print('set System state stopped.')
du2.set_system_state(du2.stopped)
print('System state:', du2.get_system_state())
time.sleep(1)
print('Power     (W)  :', du2.get_power())

print('Request Fault  :', du2.request_fault())

In [ ]:
# =========================================================
#      interactive operation of Aladdin pump controller AL-1010 (instr/al1010.py)
# =========================================================
# --- set/get inner diameter ---
D_mm = al1.get_diameter()
al1.set_diameter(D_mm)
print( f"Syringe inner diameter set to {D_mm} mm")

# --- set/get pumping rate ---
rate, unit = al1.get_pumping_rate()
al1.set_pumping_rate(rate, unit)
print( f"Pumping rate set to {rate} {unit} ('UM'=uL/min, 'MM'=mL/min, 'UH'=uL/hr, 'MH'=mL/hr)")

# --- setting volume to be dispensed or continuous pumping ---
vol, unit = al1.get_volume()
al1.set_volume(vol) # 0=cont. pumping
print( f"Volume set to {vol} {unit} ('UL' microliters, 'ML' milliliters )")

# --- setting pumping direction ---
pump_dir = al1.get_pumping_direction()   # 'INF', 'WDR' for infuse, withdraw
al1.set_pumping_direction(pump_dir) # =al1.infuse, al1.withdraw, al1.reverse
#al1.set_pumping_direction(al1.infuse)
print( f"set pumping direction to {pump_dir} ('INF' infuse, 'WDR' withdraw )")

# --- start/stop pumping ---
print("Start pumping ..")
al1.start_pump()
time.sleep(1)
#time.sleep(180)
print("Stop pumping ..")
al1.stop_pump()

In [ ]:
# --- setting pumping direction ---
pump_dir = al1.withdraw   # 'INF', 'WDR' for infuse, withdraw
al1.set_pumping_direction(pump_dir) # =al1.infuse, al1.withdraw, al1.reverse
#al1.set_pumping_direction(al1.infuse)
print( f"set pumping direction to {pump_dir} ('INF' infuse, 'WDR' withdraw )")

al1.set_pumping_rate(0.7, "MM")

# --- start/stop pumping ---
print("Start pumping ..")
al1.start_pump()
time.sleep(10)
#time.sleep(180)
print("Stop pumping ..")
al1.stop_pump()

In [5]:
# =========================================================
#      interactive operation of Präzitherm TR28-3T-RS hotplate (instr/cal3300.py)
# =========================================================
N = 1
if False: # --- set target temperature ---
    target_T_C = 40.0
    T_old = c33.read_setpoint()
    c33.set_setpoint(target_T_C)
    print( f"old setpoint : {T_old} C  -> new setpoint : {target_T_C} C" )
    N = 20
    
t00 =time.time()
for i in range(N):
    S = c33.read_setpoint()
    time.sleep(0.02)
    T = c33.readT_C()
    print( f"{time.time() - t00:6.2f} s  setpoint : {S} C  ; temperature : {T} C" )
    time.sleep(0.5)

  0.07 s  setpoint : 40.0 C  ; temperature : 39.9 C


In [13]:
# =========================================================
#      interactive operation of stepper motor PD4-N5918 M4204 from NanoTec (instr/pd4n5918.py)
# =========================================================
if True: # --- move to position ---
    end_mm, speed_mm_s, acc_mm_s2 = 0, 10.0, 5.0 # (-2) 0..346 (348) mm
    pos_mm = pd4.get_current_pos_abs_mm()
    print(f"current pos {pos_mm} mm ; target pos {end_mm} mm ; speed {speed_mm_s} mm/s : acceleration {acc_mm_s2} mm/s2 ..  (stop motor with: pd4.stop() ; set position with: pd4.set_is_at_pos_abs_mm(pos)")
    t00 = time.time()
    pd4.move_to_pos_abs_mm(end_mm, speed_mm_s, acc_mm_s2)
    while not pd4.ready():
        time.sleep(0.02)
    print(f"Dt = {time.time() - t00:.3f} s")
if False: # --- set position ---
    pd4.set_is_at_pos_abs_mm(0) # [mm]
# -- get status and position ---
mot_stat = pd4.Status_auslesen(pd4.mid)
pos_mm = pd4.get_current_pos_abs_mm()
print(f"motor {pd4.mid}: status={mot_stat:X}, position = {pos_mm} mm")
print("finished.")

current pos 20.0 mm ; target pos 0 mm ; speed 10.0 mm/s : acceleration 5.0 mm/s2 ..  (stop motor with: pd4.stop() ; set position with: pd4.set_is_at_pos_abs_mm(pos)
Dt = 3.886 s
motor 1: status=A1, position = 0.0 mm
finished.


In [ ]:
pd4.stop()

In [ ]:
f"{1500/(496-109):.2f} um/pxl", 0.61*0.45 / (25.4/(2*150))

# Program description
## Concept
### Instruments
 - Sonozap Digital Ultrasonic Generator    (instr/dug2005.py)
 - Aladdin pump controller AL-1010     (instr/al1010.py)
 - Präzitherm TR28-3T-RS hotplate     (instr/cal3300.py)
 - stepper motor PD4-N5918 M4204 from NanoTec     (instr/pd4n5918.py)

###  Program functionality
 - initialization of instruments
 - interactive operation of individual instruments (for testing)
 - deposition run with given parameters
 
 individual functionality is accessable in individuall jupyter notebook cells
 
### Technical memos
- check git status (in ~/py/meas/ ):
~~~
    git status
~~~

- store to git server (in ~/py/meas/ ):
~~~
    git add ...
    git commit -m "..cmt.."
    git push origin main
~~~
- 
get latest from git server (in ~/py/meas/ ):
~~~
    git pull
~~~
